<a href="https://colab.research.google.com/github/PankajaLaks/LT-LBP/blob/main/LT_LBP_compute.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.layers import Input,Conv2D,AveragePooling2D,Dense,Flatten
from tensorflow.keras.layers import concatenate,Embedding
from tensorflow.keras.utils  import plot_model,to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score,roc_curve,confusion_matrix,precision_score,recall_score,f1_score,roc_auc_score
import os

In [ ]:
import matplotlib.pyplot as plt
import pickle
import random
from sklearn.model_selection import train_test_split

import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.filters.rank import entropy
from skimage.morphology import disk

In [ ]:
def get_pixel(img,center,x,y):

    new_value=0

    try:
        if img[x][y]>=center:
            new_value=1
    except:
        pass
    return new_value

def lbp_calculated_pixel(img,x,y):
    center=img[x][y]
    val_ar=[]

    #Top_left

    val_ar.append(get_pixel(img,center,x-1,y-1))

    #Top

    val_ar.append(get_pixel(img,center,x-1,y))

    #top_right

    val_ar.append(get_pixel(img,center,x-1,y+1))

    #right

    val_ar.append(get_pixel(img,center,x,y+1))

    #bottom_right

    val_ar.append(get_pixel(img,center,x+1,y+1))

    #bottom

    val_ar.append(get_pixel(img,center,x+1,y))

    # bottom_left
    val_ar.append(get_pixel(img, center, x + 1, y-1))

    # left
    val_ar.append(get_pixel(img, center, x, y-1))

    # Now, we need to convert binary
    # values to decimal
    #power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    #-------------------------------
    # Computation of LT-LBP
    #-------------------------------
    power_val = [0, 1, 2, 3, 4, 5, 6, 7]
    val = 0

    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]

    return val

In [ ]:
dirpath='/content/drive/MyDrive/chestXray/'
# storepath='C:/Users/91915/LT-LBP/'
categories=['COVID','NONCOVID','PNEUMONIA']

LULB_data1=[]
LLLB_data1=[]
RULB_data1=[]
RLLB_data1=[]
training_data1=[]

height=256
weight=256


for category in categories:
    path=os.path.join(dirpath, category)
    label=categories.index(category)

    for img in os.listdir(path):
        try:
            imgpath=os.path.join(path,img)

            image=cv2.imread(imgpath,1)

            # Converting Color to Gray
            img_gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

            # Resize the image
            img_size=cv2.resize(img_gray,(256,256))


            # Applying Gaussian Smooting
            img_smooth=cv2.GaussianBlur(img_size,(5,5),cv2.BORDER_DEFAULT)

            # splitting the image into four parts
            # Left upper lobe,left lower lobe, right upper lobe, right lower lobe

            ht=img_size.shape[0]
            wth=img_size.shape[1]


            ht_cut=ht//2
            wth_cut=wth//2

            LULB1=img_smooth[:ht_cut,:wth_cut]

            LLLB1=img_smooth[ht_cut:,wth_cut:]
            RULB1=img_smooth[:wth_cut,:ht_cut]
            RLLB1=img_smooth[wth_cut:,ht_cut:]


            #Applying LT-LBP
            img_lbp1 = np.zeros((128,128),np.uint8)
            img_lbp2 = np.zeros((128,128),np.uint8)
            img_lbp3 = np.zeros((128,128),np.uint8)
            img_lbp4 = np.zeros((128,128),np.uint8)

            #LULB
            for i in range(0, 128):
                for j in range(0, 128):
                    img_lbp1[i, j] = lbp_calculated_pixel(LULB1, i, j)


            imagedata1=np.array(img_lbp1).flatten()


            LULB_data1.append([imagedata1,label])

            #LLLB
            for i in range(0, 128):
                for j in range(0, 128):
                    img_lbp2[i, j] = lbp_calculated_pixel(LLLB1, i, j)


            imagedata2=np.array(img_lbp2).flatten()

            LLLB_data1.append([imagedata2,label])

            #RULB
            for i in range(0, 128):
                for j in range(0, 128):
                    img_lbp3[i, j] = lbp_calculated_pixel(RULB1, i, j)


            imagedata3=np.array(img_lbp3).flatten()

            RULB_data1.append([imagedata3,label])

            #RLLB
            for i in range(0, 128):
                for j in range(0, 128):
                    img_lbp4[i, j] = lbp_calculated_pixel(RLLB1, i, j)


            imagedata4=np.array(img_lbp4).flatten()

            RLLB_data1.append([imagedata4,label])
        except Exception as e:
            pass



print(len(LULB_data1))

print(len(LLLB_data1))

print(len(RULB_data1))

print(len(RLLB_data1))

10291
10291
10291
10291
